
# BĐS → CSV 
Luồng dữ liệu end-to-end
1. Cấu hình tham số, selector, UA.
2. Crawl danh sách:
- Mở trình duyệt → truy cập trang p{START_PAGE} → scroll/đợi/consent → trích link.
- Lặp qua N_PAGES, chuyển trang, tiếp tục trích link (loại trùng).
3. Crawl chi tiết:
- Mở mỗi link → lấy HTML → regex-first để trích 9 cột → cộng dồn vào rows.
- Lỗi nhiều lần → thêm dòng trống để giữ dấu link.
4. Hợp nhất CSV:
- Nếu có CSV cũ và APPEND_MODE=True → gộp theo link, ưu tiên dữ liệu mới không rỗng.
5. Xuất bds.csv và xem mẫu.

## 0) Cài thư viện
Cài xong thư viện thì nhấn restart rồi từ lần sau không cần chạy cell này nữa

In [ ]:
%pip install -U pip setuptools wheel
%pip install undetected-chromedriver selenium beautifulsoup4 lxml pandas

## 1) Cấu hình
- BASE_URLS muốn lấy thêm nhiều chỗ khác thì sửa hoặc thêm chỗ này
- START_PAGE: Bắt đầu từ trang nào
- N_PAGES: muốn nó lấy bao nhiêu trang 
> Còn lại không phải chỉnh gì

In [ ]:
import re, time, random
from pathlib import Path
from typing import Dict, Optional
import pandas as pd
from bs4 import BeautifulSoup

# Common selectors and browser settings
CARD_SEL   = "div.re__card, div.property-item, div.js__card"
ANCHOR_SEL = "a[href*='/ban-'], a[href*='/nha-dat-'], a[href*='/can-ho-'], a[href*='/dat-']"
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"

BASE_URLS = {
    "Bán nhà đất hà nội": "https://batdongsan.com.vn/nha-dat-ban-ha-noi",
}
BASE_URL   = BASE_URLS["Bán nhà đất hà nội"]
START_PAGE = 1
N_PAGES    = 1
OUTPUT_CSV = "bds.csv"
APPEND_MODE = True
DELAY_RANGE = (3, 4)
HEADLESS = False  # khuyến nghị để giảm chặn; bật True nếu cần chạy nền
VERBOSE_DETAIL = True  # tắt/mở log chi tiết

COLS9 = ["ngay_dang","loai_hinh","dien_tich","gia","giay_to_phap_ly","so_phong_ngu","so_phong_ve_sinh","tinh_trang_noi_that","link"]

def text_clean(x: Optional[str]) -> str:
    return re.sub(r"\s+", " ", x or "").strip()

def pause(a: Optional[float] = None, b: Optional[float] = None) -> None:
    """Ngủ ngẫu nhiên trong khoảng [a,b]; nếu không truyền thì dùng DELAY_RANGE."""
    if a is None or b is None:
        a, b = DELAY_RANGE
    time.sleep(random.uniform(a, b))

Chọn danh mục:

- BASE_URLS liệt kê nhiều chuyên mục BĐS.
- BASE_URL = BASE_URLS["Bán chung cư mini"]
- 
Các tham số chạy:

- OUTPUT_CSV = "bds.csv"
- APPEND_MODE = True (gộp vào CSV cũ)
- DELAY_RANGE = (3,4) giây (sleep ngẫu nhiên)
- HEADLESS = False (chạy hiển thị để giảm chặn)
- VERBOSE_DETAIL = True (in log chi tiết pha chi tiết)
Tiện ích:

- text_clean(x) chuẩn hóa khoảng trắng.
- pause(a,b) ngủ ngẫu nhiên (mặc định lấy DELAY_RANGE).
> Ý nghĩa: Cell này cấu hình toàn bộ tham số, selector, và tiện ích chung.

## 2) Lấy link ở danh sách (Selenium headful)

In [3]:
def to_abs_link(href: str) -> str:
    if not href:
        return ""
    return href if href.startswith("http") else ("https://batdongsan.com.vn" + href)


def parse_list_page(html: str):
    soup = BeautifulSoup(html, "lxml")
    cards = soup.select(CARD_SEL)
    out = []
    if cards:
        for c in cards:
            a = c.select_one(ANCHOR_SEL)
            if not a:
                continue
            href = a.get("href", "")
            link = to_abs_link(href)
            if link:
                out.append({"link": link})
        return out
    # Fallback: đi thẳng theo anchor nếu không bắt được card
    for a in soup.select(ANCHOR_SEL):
        href = a.get("href", "")
        link = to_abs_link(href)
        if link:
            out.append({"link": link})
    return out

parse_list_page(html):
- Parse HTML với BeautifulSoup("lxml").
- Ưu tiên bắt theo CARD_SEL; trong mỗi card tìm anchor theo ANCHOR_SEL.
- Nếu không bắt được card, fallback: quét trực tiếp các anchor ANCHOR_SEL toàn trang.
- Trả về list dict: [{ "link": "..." }, ...].
> Ý nghĩa: Chỉ thu link từ HTML (HTML sẽ được lấy từ Selenium ở bước sau).

## 3) Selenium điều hướng

In [4]:
def human_scroll(driver, steps=8, bottom_pause=(0.8, 1.6)):
    h = driver.execute_script("return document.body.scrollHeight")
    y = 0
    for _ in range(steps):
        y += max(120, h // steps)
        driver.execute_script(f"window.scrollTo(0, {int(y)});")
        pause(0.4, 0.9)
    pause(*bottom_pause)


def click_consent_if_any(driver):
    texts = ["Đồng ý", "Chấp nhận", "Cho phép", "Tôi hiểu", "Accept", "OK", "Got it"]
    try:
        for b in driver.find_elements("css selector", "button, .btn, [role='button']"):
            t = (b.text or "").strip()
            if any(x.lower() in t.lower() for x in texts):
                try:
                    b.click()
                    pause(0.7, 1.0)
                    break
                except:
                    pass
    except:
        pass


def wait_for_cards(driver, timeout=25):
    t0 = time.time()
    while time.time() - t0 < timeout:
        n_cards = driver.execute_script(
            "return document.querySelectorAll(arguments[0]).length;",
            CARD_SEL,
        )
        n_anchors = driver.execute_script(
            "return document.querySelectorAll(arguments[0]).length;",
            ANCHOR_SEL,
        )
        if (n_cards and n_cards > 0) or (n_anchors and n_anchors > 0):
            return True
        pause(0.9, 1.2)
    return False


def goto_page(driver, base_url: str, page_index: int):
    url = base_url if page_index <= 1 else f"{base_url}/p{page_index}"
    driver.get(url)


def click_next_page(driver, current_page_idx: int, base_url: str):
    tried = False
    try:
        for a in driver.find_elements(
            "css selector", "a[rel='next'], a[aria-label*='Sau'], a[title*='Sau'], li a"
        ):
            t = (a.get_attribute("aria-label") or "") + " " + (
                a.get_attribute("title") or ""
            ) + " " + (a.text or "")
            if any(x in t.lower() for x in ["sau", "next", "tiếp", ">", "»"]):
                a.click()
                tried = True
                break
    except:
        pass
    if not tried:
        goto_page(driver, base_url, current_page_idx + 1)


def create_chrome(headless: bool):
    import undetected_chromedriver as uc
    from selenium.webdriver.chrome.options import Options

    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--window-size=1366,768")
    opts.add_argument("--lang=vi-VN")
    opts.add_argument(f"--user-agent={USER_AGENT}")
    return uc.Chrome(options=opts)


def crawl_list_links(n_pages: int, base_url: str, start_page: int):
    driver = create_chrome(HEADLESS)
    results = []
    seen = set()
    try:
        goto_page(driver, base_url, start_page)
        pause()
        click_consent_if_any(driver)
        human_scroll(driver, steps=10)
        wait_for_cards(driver, timeout=25)
        current_idx = start_page
        for i in range(n_pages):
            # Thử tối đa 2 lần nếu trang hiện tại chưa bắt được link
            attempts = 2
            page_items = []
            for _ in range(attempts):
                html = driver.page_source
                page_items = parse_list_page(html)
                if page_items:
                    break
                # scroll sâu thêm rồi đợi
                human_scroll(driver, steps=12)
                pause(1.0, 1.6)
                wait_for_cards(driver, timeout=10)
            added_this_page = 0
            for it in page_items:
                link = it["link"]
                if link not in seen:
                    results.append({"link": link})
                    seen.add(link)
                    added_this_page += 1
            print(
                f"[List Page {current_idx}] links (raw): {len(page_items)} | unique_added: {added_this_page} | unique_total: {len(results)}"
            )
            if len(page_items) == 0:
                print(
                    "[Hint] 0 link: trang có thể bị chặn/che overlay. Hãy thử HEADLESS=False (đã để mặc định) hoặc tăng DELAY_RANGE."
                )
            # Nếu còn trang phải xử lý thì mới chuyển trang
            if i < n_pages - 1:
                click_next_page(
                    driver, current_page_idx=current_idx, base_url=base_url
                )
                current_idx += 1
                pause()
                click_consent_if_any(driver)
                human_scroll(driver, steps=8)
                wait_for_cards(driver, timeout=20)
    finally:
        driver.quit()
    return results

1. Tiện ích “giống người”:
- human_scroll(driver, steps, bottom_pause): cuộn trang nhiều lần kèm pause ngắn.
- click_consent_if_any(driver): cố gắng bấm các nút “đồng ý/accept/OK…” để vượt cookie/overlay.
- wait_for_cards(driver, timeout): đợi đến khi thấy card/anchor xuất hiện (querySelectorAll với CARD_SEL và ANCHOR_SEL).
2. Điều hướng/trang:
- goto_page(driver, base_url, page_index): chuyển tới trang danh sách p{index}.
- click_next_page(driver, current_page_idx, base_url): cố gắng bấm next; nếu không được thì fallback goto_page p{+1}.
3. Tạo Chrome thống nhất:
- Mục tiêu: giảm bị chặn và đồng nhất cấu hình giữa các pha.
4. Hàm chính crawl danh sách:
- crawl_list_links(n_pages, base_url, start_page):
- Vào trang bắt đầu, pause, xử lý consent, scroll, đợi card.
- Với mỗi trang (lặp N_PAGES):
- Thử tối đa 2 lần parse link (lấy driver.page_source → parse_list_page).
- Nếu chưa có link: scroll sâu hơn, pause thêm, đợi ngắn.
- Loại trùng qua seen (set) và đếm số link mới thêm ở trang hiện tại (unique_added).
- Log: “[List Page i] links (raw): … | unique_added: … | unique_total: …”
- Nếu còn trang phải lấy thì chuyển trang next, pause/consent/scroll/wait.
5. Trả về results: list các dict {"link": ...} đã loại trùng.
>Ý nghĩa: Thu thập link từ nhiều trang danh sách với hành vi “giống người” để hạn chế bị chặn.


## 4) Regex-first trên section “Đặc điểm bất động sản”

In [5]:
BAD_VALUES = {"tai ung dung","tải ứng dụng","xem thêm trên app","app","ứng dụng"}

def get_breadcrumb_type(soup: BeautifulSoup) -> str:
    for sel in ["nav[aria-label*='breadcrumb'] a", ".re__breadcrumb a", "ol.breadcrumb a"]:
        links = [text_clean(a.get_text()) for a in soup.select(sel) if text_clean(a.get_text())]
        if links:
            for txt in reversed(links):
                if "căn hộ" in txt.lower(): return txt
            return links[-1]
    a = soup.select_one("a.re__link-se")
    if a:
        txt = text_clean(a.get_text())
        if "căn hộ" in txt.lower(): return txt
    a2 = soup.select_one("nav a:last-child, ol.breadcrumb li:last-child a")
    return text_clean(a2.get_text()) if a2 else ""

def get_characteristics_text(soup: BeautifulSoup) -> str:
    # tìm section có 'Đặc điểm bất động sản', nếu không thấy dùng toàn trang
    for sec in soup.select("section, .re__section, .re__pr-specs"):
        head = sec.find(["h2","h3","h4"])
        if head and "đặc điểm" in text_clean(head.get_text()).lower():
            return sec.get_text("\n", strip=True)
    return soup.get_text("\n", strip=True)

def extract_fields_from_text(txt: str) -> dict:
    # Chuẩn hóa khoảng trắng nhưng GIỮ xuống dòng để bắt đầu dòng bằng nhãn
    t = re.sub(r"[ \t]+", " ", txt)

    def find_line_value(label_pattern: str) -> str:
        m = re.search(rf"^(?:{label_pattern})\s*[:\-]?\s*([^\n\r]+)", t, flags=re.I | re.M)
        return text_clean(m.group(1)) if m else ""

    # Giá (ưu tiên nhãn chính xác ở đầu dòng)
    gia = find_line_value(r"Khoảng\s*giá|Mức\s*giá|Giá")

    # Diện tích (dạng '70 m²' giữ nguyên số thập phân dùng dấu phẩy)
    m = re.search(r"^Diện\s*tích\s*[:\-]?\s*([0-9\.,]+ ?m²)", t, flags=re.I | re.M)
    dien_tich = text_clean(m.group(1)) if m else ""

    # Phòng ngủ
    pn_raw = find_line_value(r"Số\s*phòng\s*ngủ")

    # WC
    m_wc = re.search(r"^Số\s*phòng\s*tắm,\s*vệ\s*sinh\s*[:\-]?\s*([0-9]+)", t, flags=re.I | re.M)
    wc_raw = m_wc.group(1) if m_wc else find_line_value(r"(?:WC|Vệ\s*sinh|Toilet|Phòng\s*tắm)")

    # Nội thất
    noi_that_raw = find_line_value(r"Nội\s*thất")
    noi_that = ""
    if noi_that_raw:
        m2 = re.search(r"(đầy\s*đủ|full\s*nội\s*thất|cơ\s*bản|trống|không\s*nội\s*thất)", noi_that_raw, flags=re.I)
        noi_that = text_clean(m2.group(0)) if m2 else noi_that_raw

    # Pháp lý
    giay_to = find_line_value(r"(?:Pháp\s*lý|Giấy\s*tờ(?:\s*pháp\s*lý)?)")
    if any(bad in giay_to.lower() for bad in BAD_VALUES):
        giay_to = ""

    # Làm sạch 'phòng' khỏi số lượng
    def only_digits(s: str) -> str:
        m = re.search(r"\d+", s or "")
        return m.group(0) if m else (s or "").strip()

    so_pn = only_digits(pn_raw)
    so_wc = only_digits(wc_raw)

    # Loại bỏ giá trị giá không hợp lệ (ví dụ 'Biểu đồ giá')
    if gia and any(x in gia.lower() for x in {"biểu đồ giá", "liên hệ", "đang cập nhật"}):
        gia = ""

    return {
        "gia": gia,
        "dien_tich": dien_tich,
        "so_phong_ngu": so_pn,
        "so_phong_ve_sinh": so_wc,
        "tinh_trang_noi_that": noi_that,
        "giay_to_phap_ly": giay_to,
    }

def extract_ngay_dang(soup: BeautifulSoup) -> str:
    for node in soup.select("section, .re__pr-time, .re__pr-config, .re__pr-attribute, [class*='time'], [class*='date']"):
        txt = text_clean(node.get_text(" ", strip=True))
        if "ngày đăng" in txt.lower():
            m = re.search(r"(\d{1,2}/\d{1,2}/\d{4})", txt)
            if m: return m.group(1)
    m = re.search(r"(\d{1,2}/\d{1,2}/\d{4})", soup.get_text(" ", strip=True))
    return m.group(1) if m else ""

def extract_detail_v8(html: str, link: str) -> Dict[str, str]:
    soup = BeautifulSoup(html, "lxml")
    sec_text = get_characteristics_text(soup)
    fields = extract_fields_from_text(sec_text)
    return {
        "ngay_dang": extract_ngay_dang(soup),
        "loai_hinh": get_breadcrumb_type(soup),
        "dien_tich": fields["dien_tich"],
        "gia": fields["gia"],
        "giay_to_phap_ly": fields["giay_to_phap_ly"],
        "so_phong_ngu": fields["so_phong_ngu"],
        "so_phong_ve_sinh": fields["so_phong_ve_sinh"],
        "tinh_trang_noi_that": fields["tinh_trang_noi_that"],
        "link": link,
    }

1. extract_fields_from_text(txt):
- Chuẩn hóa khoảng trắng nhưng giữ xuống dòng để match “nhãn đầu dòng”.
- find_line_value(label_pattern) bắt các dòng dạng “Nhãn: Giá trị”.
2. Bắt các trường:
- gia: “Khoảng giá | Mức giá | Giá” (lọc ra các giá trị phi cấu trúc như “Liên hệ”, “Biểu đồ giá” → để rỗng).
- dien_tich: “Diện tích: 70 m²” (khớp số và m²).
- so_phong_ngu: “Số phòng ngủ”.
- so_phong_ve_sinh: “Số phòng tắm, vệ sinh” hoặc WC/Toilet/…
- tinh_trang_noi_that: “Nội thất” (cố gắng chuẩn hóa: “đầy đủ”, “cơ bản”, “trống”…).
- giay_to_phap_ly: “Pháp lý | Giấy tờ (pháp lý)” (lọc giá trị rác).
- only_digits bóc số từ chuỗi (ví dụ “2 phòng ngủ” → “2”).
3. extract_ngay_dang(soup):
- Tìm “ngày đăng” theo regex dd/mm/yyyy trong các vùng time/config/attribute; fallback: toàn trang.
4. extract_detail_v8(html, link):
- Parse soup, lấy text đặc điểm, gọi extract_fields_from_text, ghép với ngay_dang, loai_hinh, và link.
- Trả về dict đủ 9 cột.
> Ý nghĩa: Ưu tiên regex theo nhãn để bền vững với thay đổi DOM.

## 5) Crawl → chi tiết → 9 cột

In [6]:
def crawl_v8(n_pages: int, base_url: str, start_page: int) -> pd.DataFrame:
    links = crawl_list_links(n_pages, base_url, start_page)
    if not links: return pd.DataFrame(columns=COLS9)

    if VERBOSE_DETAIL:
        print(f"[Detail] Tổng link sẽ crawl: {len(links)}")

    driver = create_chrome(HEADLESS)

    rows = []
    try:
        total = len(links)
        for idx, it in enumerate(links, start=1):
            url = it["link"]
            success = False
            for attempt in range(1, 3):  # 2 attempts
                try:
                    driver.get(url)
                    pause(1.8, 2.8)
                    html = driver.page_source
                    rows.append(extract_detail_v8(html, link=url))
                    if VERBOSE_DETAIL:
                        print(f"[Detail {idx}/{total}] OK")
                    pause(0.8, 1.4)
                    success = True
                    break
                except Exception as e:
                    if VERBOSE_DETAIL:
                        print(f"[Detail {idx}/{total}] Retry {attempt} error: {e}")
                    pause(1.2, 2.0)
            if not success:
                miss = {k:"" for k in COLS9}; miss["link"] = url; rows.append(miss)
                if VERBOSE_DETAIL:
                    print(f"[Detail {idx}/{total}] FAIL -> added empty row")
    finally:
        driver.quit()

    return pd.DataFrame(rows, columns=COLS9)

crawl_v8(n_pages, base_url, start_page) -> DataFrame:
- Gọi crawl_list_links để lấy link.
- Log tổng số link nếu VERBOSE_DETAIL.
- driver = create_chrome(HEADLESS).
- Lặp từng link
- Trả về DataFrame(rows, columns=COLS9).
> Ý nghĩa: Vào chi tiết từng tin để trích đủ 9 cột theo chuẩn.

## 6) Append theo link (ưu tiên dữ liệu mới không rỗng)
>Ý nghĩa: Điểm vào chính để chạy toàn bộ pipeline và lưu CSV.

In [7]:

def smart_append_by_link(old_df: pd.DataFrame, new_df: pd.DataFrame) -> pd.DataFrame:
    for c in COLS9:
        if c not in old_df.columns: old_df[c] = ""
        if c not in new_df.columns: new_df[c] = ""
    old_df = old_df[COLS9].drop_duplicates(subset=["link"], keep="first")
    new_df = new_df[COLS9].drop_duplicates(subset=["link"], keep="first")
    both = pd.concat([old_df, new_df], ignore_index=True).sort_values("link")

    def pick_nonempty(series):
        for v in series[::-1]:
            if pd.notna(v) and str(v).strip() != "":
                return v
        return series.iloc[-1]

    merged = both.groupby("link", as_index=False).agg({c: pick_nonempty for c in COLS9})
    return merged[COLS9]


smart_append_by_link(old_df, new_df):
- Đảm bảo đủ cột, loại trùng theo link trong mỗi df.
- Ghép old_df + new_df, sort theo link.
- Gộp theo link, với mỗi cột chọn “giá trị không rỗng cuối cùng trong nhóm” → ưu tiên dữ liệu mới không rỗng (vì new nằm sau).
- Trả về merged[COLS9].
> Ý nghĩa: Update dữ liệu cho link cũ bằng dữ liệu mới nếu có.

## 7) RUN ALL

In [8]:
print("BASE_URL:", BASE_URL, "| START_PAGE:", START_PAGE, "| N_PAGES:", N_PAGES)
df_new = crawl_v8(N_PAGES, BASE_URL, START_PAGE)
print("Số dòng mới:", len(df_new))

csv_path = Path(OUTPUT_CSV)
if APPEND_MODE and csv_path.exists():
    old_df = pd.read_csv(csv_path)
    final_df = smart_append_by_link(old_df, df_new)
else:
    final_df = df_new

final_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"✅ Đã lưu CSV: {OUTPUT_CSV} (tổng {len(final_df)} dòng)")
display(final_df.head(20))


BASE_URL: https://batdongsan.com.vn/ban-can-ho-chung-cu-mini | START_PAGE: 1 | N_PAGES: 1
[List Page 1] links (raw): 20 | unique_added: 20 | unique_total: 20
[List Page 1] links (raw): 20 | unique_added: 20 | unique_total: 20
[Detail] Tổng link sẽ crawl: 20
[Detail] Tổng link sẽ crawl: 20
[Detail 1/20] OK
[Detail 1/20] OK
[Detail 2/20] OK
[Detail 2/20] OK
[Detail 3/20] OK
[Detail 3/20] OK
[Detail 4/20] OK
[Detail 4/20] OK
[Detail 5/20] OK
[Detail 5/20] OK
[Detail 6/20] OK
[Detail 6/20] OK
[Detail 7/20] OK
[Detail 7/20] OK
[Detail 8/20] OK
[Detail 8/20] OK
[Detail 9/20] OK
[Detail 9/20] OK
[Detail 10/20] OK
[Detail 10/20] OK
[Detail 11/20] OK
[Detail 11/20] OK
[Detail 12/20] OK
[Detail 12/20] OK
[Detail 13/20] OK
[Detail 13/20] OK
[Detail 14/20] OK
[Detail 14/20] OK
[Detail 15/20] OK
[Detail 15/20] OK
[Detail 16/20] OK
[Detail 16/20] OK
[Detail 17/20] OK
[Detail 17/20] OK
[Detail 18/20] OK
[Detail 18/20] OK
[Detail 19/20] OK
[Detail 19/20] OK
[Detail 20/20] OK
[Detail 20/20] OK
Số dòng 

,ngay_dang,loai_hinh,dien_tich,gia,giay_to_phap_ly,so_phong_ngu,so_phong_ve_sinh,tinh_trang_noi_that,link
0,31/10/2025,"Chung cư mini, căn hộ dịch vụ tại đường Đại Từ",48 m²,"1,88 tỷ",Sổ đỏ/ Sổ hồng,2,1,Đầy đủ,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
1,31/10/2025,"Chung cư mini, căn hộ dịch vụ tại đường Tam Trinh",120 m²,"33,4 tỷ",Sổ đỏ/ Sổ hồng,33,33,,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
2,30/10/2025,"Chung cư mini, căn hộ dịch vụ tại đường Nguyễn...",40 m²,"1,58 tỷ",Sổ đỏ/ Sổ hồng,1,1,Không nội thất,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
3,30/10/2025,"Chung cư mini, căn hộ dịch vụ tại Diamond Gold...",137 m²,45 triệu/m²,Sổ đỏ/ Sổ hồng,,,Cơ bản,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
4,28/10/2025,"Chung cư mini, căn hộ dịch vụ tại đường Tây Mỗ",100 m²,26 tỷ,Sổ đỏ/ Sổ hồng,30,30,Đầy đủ,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
5,29/10/2025,"Chung cư mini, căn hộ dịch vụ tại phố Thái Hà",38 m²,"1,65 tỷ",Hợp đồng mua bán vi bằng,1,1,Đầy đủ,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
6,29/10/2025,"Chung cư mini, căn hộ dịch vụ tại đường Phạm Hùng",35 m²,"1,25 tỷ",Sổ đỏ/ Sổ hồng,2,1,Đầy đủ,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
7,28/10/2025,"Chung cư mini, căn hộ dịch vụ tại đường Tây Tựu",40 m²,950 triệu,Hợp đồng mua bán,2,1,Đầy đủ,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
8,27/10/2025,"Chung cư mini, căn hộ dịch vụ tại phường Mỹ Đì...",30 m²,"1,25 tỷ",", ký HDMB, giấy phép xây dựng, tách sổ hồng th...",1,1,Đầy đủ,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
9,27/10/2025,"Chung cư mini, căn hộ dịch vụ tại phường Trung...",37 m²,"1,29 tỷ",Tách sổ hồng vĩnh viễn.,1,1,Đầy đủ,https://batdongsan.com.vn/ban-can-ho-chung-cu-...
